![Vespa logo](https://vespa.ai/assets/vespa-logo-color.png)

# Pyvespa examples

This is a notebook with short examples one can build applications from.

Refer to [troubleshooting](https://pyvespa.readthedocs.io/en/latest/troubleshooting.html), which also has utilies for debugging.

In [ ]:
!pip3 install pyvespa

## Neighbors

Explore distance between points in 3D vector space.

This is a simple example feeding documents with a tensor representing a point in space,
and a rank profile calculating the distance between a point in the query and the point in the documents.

### Distant neighbor

First, _find the point that is  **most**  distant from a point in query,
using  [euclidean-distance](https://docs.vespa.ai/en/reference/ranking-expressions.html#euclidean-distance-t)._

Deploy the Application Package:

In [8]:
from vespa.package import ApplicationPackage, Field, RankProfile
from vespa.deployment import VespaDocker

app_package = ApplicationPackage(name="neighbors")

app_package.schema.add_fields(
    Field(name = "point", type = "tensor<float>(d[3])", indexing = ["attribute", "summary"])
)

app_package.schema.add_rank_profile(
    RankProfile(
        name = "max_distance",
        inputs = [("query(qpoint)", "tensor<float>(d[3])")],
        first_phase = "euclidean_distance(attribute(point), query(qpoint), d)"
    )
)

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Waiting for application status, 10/300 seconds...
Waiting for application status, 15/300 seconds...
Waiting for application status, 20/300 seconds...
Waiting for application status, 25/300 seconds...
Finished deployment.


Feed points in 3d space using a 3-dimensional [indexed tensor](https://docs.vespa.ai/en/tensor-user-guide.html).
Pyvespa feeds using the [/document/v1/ API](https://docs.vespa.ai/en/reference/document-v1-api-reference.html), refer to [document format](https://docs.vespa.ai/en/reference/document-json-format.html):

In [16]:
feed = [
        {
            'id': 0,
            'fields': {
                'point': [0.0, 1.0, 2.0]
            }
        },
        {
            'id': 1,
            'fields': {
                'point': [1.0, 2.0, 3.0]
            }
        },
        {
            'id': 2,
            'fields': {
                'point': [2.0, 3.0, 4.0]
            }
        }
]

response = app.feed_batch(schema="neighbors", batch=feed, asynchronous=False)

Successful documents fed: 3/3.
Batch progress: 1/1.


Query from origo using [YQL](https://docs.vespa.ai/en/query-language.html).
The rank profile will rank the most distant points highest, here `sqrt(2*2 + 3*3 + 4*4) = 5.385`:

In [17]:
import json
result = app.query(body={
  'yql': 'select point from neighbors where true',
  'input.query(qpoint)': '[0.0, 0.0, 0.0]',
  'ranking.profile': 'max_distance'
})
print(json.dumps(result.hits, indent=4))

[
    {
        "id": "index:neighbors_content/0/c81e728dfde15fa4e8dfb3d3",
        "relevance": 5.385164807134504,
        "source": "neighbors_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    2.0,
                    3.0,
                    4.0
                ]
            }
        }
    },
    {
        "id": "index:neighbors_content/0/c4ca4238db266f395150e961",
        "relevance": 3.7416573867739413,
        "source": "neighbors_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    1.0,
                    2.0,
                    3.0
                ]
            }
        }
    },
    {
        "id": "index:neighbors_content/0/cfcd20845b10b1420c6cdeca",
        "relevance": 2.23606797749979,
        "source": "neighbors_content",
        "fields": {
            "point": {
              

Query from `[1.0, 2.0, 2.9]` - find that `[2.0, 3.0, 4.0]` is most distant:

In [18]:
result = app.query(body={
  'yql': 'select point from neighbors where true',
  'input.query(qpoint)': '[1.0, 2.0, 2.9]',
  'ranking.profile': 'max_distance'
})
print(json.dumps(result.hits, indent=4))

[
    {
        "id": "index:neighbors_content/0/c81e728dfde15fa4e8dfb3d3",
        "relevance": 1.7916472308265357,
        "source": "neighbors_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    2.0,
                    3.0,
                    4.0
                ]
            }
        }
    },
    {
        "id": "index:neighbors_content/0/cfcd20845b10b1420c6cdeca",
        "relevance": 1.6763055154708881,
        "source": "neighbors_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    0.0,
                    1.0,
                    2.0
                ]
            }
        }
    },
    {
        "id": "index:neighbors_content/0/c4ca4238db266f395150e961",
        "relevance": 0.09999990575011103,
        "source": "neighbors_content",
        "fields": {
            "point": {
          

### Nearest neighbor

The [nearestNeighbor](https://docs.vespa.ai/en/reference/query-language-reference.html#nearestneighbor) query operator
calculates distances between points in vector space,
and the [closeness()](https://docs.vespa.ai/en/reference/rank-features.html#closeness(dimension,name)) rank feature can be used to rank results:

In [19]:
app_package.schema.add_rank_profile(
    RankProfile(
        name = "nearest_neighbor",
        inputs = [("query(qpoint)", "tensor<float>(d[3])")],
        first_phase = "closeness(field, point)"
    )
)

app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Waiting for application status, 0/300 seconds...
Waiting for application status, 5/300 seconds...
Finished deployment.


Read more in [nearest neighbor search](https://docs.vespa.ai/en/nearest-neighbor-search.html).

Query using nearestNeighbor query operator:

In [20]:
result = app.query(body={
  'yql': 'select point from neighbors where {targetHits: 3}nearestNeighbor(point, qpoint)',
  'input.query(qpoint)': '[1.0, 2.0, 2.9]',
  'ranking.profile': 'nearest_neighbor'
})
print(json.dumps(result.hits, indent=4))

[
    {
        "id": "index:neighbors_content/0/c4ca4238db266f395150e961",
        "relevance": 0.9090909879069752,
        "source": "neighbors_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    1.0,
                    2.0,
                    3.0
                ]
            }
        }
    },
    {
        "id": "index:neighbors_content/0/cfcd20845b10b1420c6cdeca",
        "relevance": 0.37364941905256455,
        "source": "neighbors_content",
        "fields": {
            "point": {
                "type": "tensor<float>(d[3])",
                "values": [
                    0.0,
                    1.0,
                    2.0
                ]
            }
        }
    },
    {
        "id": "index:neighbors_content/0/c81e728dfde15fa4e8dfb3d3",
        "relevance": 0.35821144946644456,
        "source": "neighbors_content",
        "fields": {
            "point": {
         

## Cleanup

In [21]:
vespa_docker.container.stop()
vespa_docker.container.remove()